In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        root = './.data',
        train = True,
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        root = './.data',
        train = False,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,),(0.3081,))
        ])        
    ),
    batch_size=BATCH_SIZE,
    shuffle=True    
)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, input):
        l_conv1 = F.relu(F.max_pool2d(self.conv1(input), 2))
        l_conv2 = F.relu(F.max_pool2d(self.drop(self.conv2(l_conv1)), 2))
        l_flatten = l_conv2.view(-1, 320)
        l_fc1 = F.relu(self.fc1(l_flatten))
#         l_drop = self.drop(l_fc1)
        l_drop = F.dropout(l_fc1, training=self.training)
        l_fc2 = self.fc2(l_drop)
        
        return F.log_softmax(l_fc2, dim=1)        

In [6]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
#         if batch_idx % 200 == 0:
#             print(f'Train epoch : {epoch} {batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 * batch_idx / len(train_loader):.0f}%)\tLoss:{loss.item():.6f}')
    

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'{epoch:2d} Test Loss : {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

 1 Test Loss : 0.6453, Accuracy: 75.21%
 2 Test Loss : 0.5830, Accuracy: 77.15%
 3 Test Loss : 0.5465, Accuracy: 78.94%
 4 Test Loss : 0.5118, Accuracy: 80.90%
 5 Test Loss : 0.4892, Accuracy: 80.57%
 6 Test Loss : 0.4828, Accuracy: 80.59%
 7 Test Loss : 0.4491, Accuracy: 83.40%
 8 Test Loss : 0.4236, Accuracy: 84.50%
 9 Test Loss : 0.4120, Accuracy: 84.87%
10 Test Loss : 0.4053, Accuracy: 85.11%
11 Test Loss : 0.3969, Accuracy: 85.42%
12 Test Loss : 0.3883, Accuracy: 85.53%
13 Test Loss : 0.3814, Accuracy: 85.98%
14 Test Loss : 0.3826, Accuracy: 85.91%
15 Test Loss : 0.3694, Accuracy: 86.50%
16 Test Loss : 0.3678, Accuracy: 86.66%
17 Test Loss : 0.3672, Accuracy: 86.45%
18 Test Loss : 0.3578, Accuracy: 86.90%
19 Test Loss : 0.3562, Accuracy: 87.07%
20 Test Loss : 0.3521, Accuracy: 87.02%
21 Test Loss : 0.3539, Accuracy: 87.26%
22 Test Loss : 0.3448, Accuracy: 87.49%
23 Test Loss : 0.3461, Accuracy: 87.35%
24 Test Loss : 0.3395, Accuracy: 87.49%
25 Test Loss : 0.3395, Accuracy: 87.48%
